In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import json
import glob
import networkx as nx
import geopandas as gpd
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from math import comb
from datetime import date
from tqdm.notebook import tqdm as tqdm_notebook
from copy import deepcopy
from itertools import islice,product
from argparse import Namespace
from functools import partial
from pyproj import Proj
from scipy.spatial import KDTree
from networkx.classes.function import path_weight
# from mpl_toolkits.basemap import Basemap as Basemap


from shapely.geometry import Point,LineString, Polygon, mapping, MultiPoint, box


from ticodm.utils import *
from ticodm.notebook_functions import *
from ticodm.spatial_interaction_model import ProductionConstrained
from ticodm.spatial_interaction_model_mcmc import ProductionConstrainedMarkovChainMonteCarlo
# from ticodm.contingency_table_mcmc import ContingencyTableMarkovChainMonteCarlo
from ticodm.contingency_table import instantiate_ct
from ticodm.sim_models.production_constrained import *

# mpl.rcParams['agg.path.chunksize'] = 10000

%matplotlib inline

# London retail

In [ ]:
# Expertiment id
dataset = 'london_retail'
origin_sizes_df = pd.read_csv('../../NeuralABM/data/HarrisWilson/London_data/origin_sizes.csv',index_col=0)
origin_sizes_arr = origin_sizes_df.values.ravel()

dest_sizes_df = pd.read_csv('../../NeuralABM/data/HarrisWilson/London_data/dest_sizes.csv',index_col=0)
dest_sizes_arr = dest_sizes_df.values.ravel()

cost_matrix_df = pd.read_csv('../../NeuralABM/data/HarrisWilson/London_data/exp_distances.csv',index_col=0)
cost_matrix_arr = -np.log(cost_matrix_df.values)

In [ ]:
# Normalise data
origin_sizes_arr_normalised = origin_sizes_arr/np.sum(origin_sizes_arr)
dest_sizes_arr_normalised = dest_sizes_arr/np.sum(dest_sizes_arr)
cost_matrix_arr_normalised = cost_matrix_arr/np.sum(cost_matrix_arr.sum())

print('Origin sizes sum',np.sum(origin_sizes_arr_normalised))
print('Destination sizes sum',np.sum(dest_sizes_arr_normalised))
print('Cost matrix sum',np.sum(cost_matrix_arr_normalised.sum()))

In [ ]:
plt.hist(cost_matrix_arr.flatten(),bins=50)
plt.show()

In [ ]:
np.savetxt('../data/inputs/london_retail/origin_sizes.txt',origin_sizes_arr)
np.savetxt('../data/inputs/london_retail/dest_sizes.txt',dest_sizes_arr)
np.savetxt('../data/inputs/london_retail/cost_mat.txt',cost_matrix_arr)
np.savetxt('../data/inputs/london_retail/origin_sizes_normalised.txt',origin_sizes_arr_normalised)
np.savetxt('../data/inputs/london_retail/dest_sizes_normalised.txt',dest_sizes_arr_normalised)
np.savetxt('../data/inputs/london_retail/cost_mat_normalised.txt',cost_matrix_arr_normalised)

In [ ]:
origin_demand = np.loadtxt('../data/inputs/london_retail/origin_supply.txt')
log_destination_attraction = np.loadtxt('../data/inputs/london_retail/log_destination_attraction.txt')
destination_attraction = np.exp(log_destination_attraction)
cost_matrix = np.loadtxt('../data/inputs/london_retail/normalised_cost_matrix.txt')
P = np.loadtxt('../data/inputs/london_retail/P.txt')
xd0 = np.loadtxt('../data/inputs/london_retail/xd0.txt')
wd0 = np.exp(xd0)
cost_mat = np.loadtxt('../data/inputs/london_retail/cost_mat.txt')

In [ ]:
for i,o in enumerate(P):
    if not np.isclose(o,origin_sizes_arr_normalised[i]):
        raise ValueError(f'NOT MATCHING {i} {o} vs {origin_sizes_arr_normalised[i]}')

In [ ]:
for i,o in enumerate(wd0):
    if not np.isclose(o,dest_sizes_arr_normalised[i]):
        raise ValueError(f'NOT MATCHING {i} {o} vs {dest_sizes_arr_normalised[i]}')

In [ ]:
for i in range(np.shape(cost_mat)[0]):
    for j in range(np.shape(cost_mat)[1]):
        if not np.isclose(cost_mat[i,j],cost_matrix_arr_normalised[i,j]):
            raise ValueError(f'NOT MATCHING {i},{j} {cost_mat[i,j]} vs {cost_matrix_arr_normalised[i,j]}')

In [ ]:
total_w = dest_sizes_arr_normalised.sum()
min_w = np.min(dest_sizes_arr_normalised)
total_o = origin_sizes_arr_normalised.sum()
M = cost_matrix_arr_normalised.shape[1]
# Compute kappa, delta
kappa = total_o / (total_w - min_w*M)
delta = kappa * min_w
print(total_o,',',total_w)
print(kappa,',',delta)

In [ ]:
total_w = wd0.sum()
min_w = np.min(wd0)
total_o = P.sum()
M = cost_mat.shape[1]
# Compute kappa, delta
kappa = total_o / (total_w - min_w*M)
delta = kappa * min_w
print(total_o,',',total_w)
print(kappa,',',delta)

In [ ]:
# Used kappa,delta
used_delta = 0.3/M
used_kappa = (total_o + used_delta*M)/total_w
print(used_kappa,',',used_delta)

In [ ]:
plot_destination_attraction_by_cost.margins[tuplize(range(self.ndims()))]
    dest_sizes_arr,
    cms=[
        cost_matrix
    ],
    cm_names=[
        f'euclidean_distance'
    ],
    fig_size=(20,5)
)